In [1]:
import math
import keras
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2025-08-14 05:47:14.044311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-14 05:47:14.072418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-14 05:47:14.081487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-14 05:47:14.175129: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    details = tf.config.experimental.get_device_details(gpu)
    compute_capability = details.get('compute_capability', 'Unknown')
    print(f"Device: {gpu.name}, Compute Capability: {compute_capability}")
    
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Device: /physical_device:GPU:0, Compute Capability: (7, 5)


I0000 00:00:1755150437.929094    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1755150438.156515    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1755150438.156845    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1755150438.157745    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
MODEL_PATH = "../model/"

In [4]:
def print_mnist(ds):
    import matplotlib.pyplot as plt

    # Function to display MNIST images
    def show_images(images, labels):
        plt.figure(figsize=(10, 5))
        for i in range(10):
            plt.subplot(2, 5, i + 1)
            plt.imshow(images[i].numpy(), cmap='gray')
            plt.title(f"Label: {labels[i].numpy()}")
            plt.axis("off")
        plt.show()

    # Display a few images from the training set
    num_images_to_display = 10
    train_images, train_labels = next(iter(ds.batch(num_images_to_display)))
    show_images(train_images, train_labels)


In [5]:
(_ds_train, _ds_test), _ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

I0000 00:00:1755150438.228411    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1755150438.228699    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1755150438.228921    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1755150438.350468    4351 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [6]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [7]:
ds_train = _ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(_ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [8]:
ds_test = _ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [9]:
model = keras.models.load_model(f'{MODEL_PATH}model.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 305,312 (1.16 MB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 203,542 (795.09 KB)

In [10]:
for t_v in model.trainable_variables:
    print("name of weight: ", t_v.name)
    print("shape of the weight", t_v.shape)
    print("value type: ", t_v.numpy().dtype)
    print()

name of weight:  kernel
shape of the weight (784, 128)
value type:  float32

name of weight:  bias
shape of the weight (128,)
value type:  float32

name of weight:  kernel
shape of the weight (128, 10)
value type:  float32

name of weight:  bias
shape of the weight (10,)
value type:  float32



In [11]:
model.trainable_variables[2].path

'dense_1/kernel'

In [12]:
for t_v in model.trainable_variables:
    name = t_v.path
    
    if "_" not in name.split("/")[0]:
        name = name.split("/")[0] + "_0/" + name.split("/")[1]
    # else:
    #     name = "/".join(name.split("/")[1:])
    
    name = name.replace("/", "_")
    
    np.save(f'{MODEL_PATH}{name}', t_v.numpy().flatten())

In [13]:
x_ls = []
y_ls = []
for x, y in iter(ds_test.unbatch()):
    x_ls.append(x)
    y_ls.append(y)

x_np = np.array(x_ls)
y_np = np.array(y_ls)

select_el = 500
print(x_np[select_el].shape, y_ls[select_el])

np.save(f"{MODEL_PATH}test_x", x_np.flatten()) # .map(lambda x: tf.cast(x, tf.float32))
np.save(f"{MODEL_PATH}test_y", y_np.flatten())

print(x_np.dtype)
print(y_np.dtype)

2025-08-14 05:47:21.961789: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(28, 28, 1) tf.Tensor(2, shape=(), dtype=int64)
float32
int64


In [14]:
import re

poly_str = "+0.0229645346261931*x**0+0.4999999999999936*x**1+1.4340123453814493*x**2+1.0966062321883419e-10*x**3-2.0891847536610593*x**4-3.4105118408214585e-10*x**5+1.1551724027032066*x**6+2.3139697111946557e-10*x**7"
level = 0

terms = re.findall(r'[+-]?\d*\.?\d+(?:e[+-]?\d+)?', poly_str)
consts = [float(term) for term in terms]
poly_consts = np.array([v for i, v in enumerate(consts) if i % 2 != 1]) #, dtype=np.float32)

np.save(f"{MODEL_PATH}poly_consts_{level}", poly_consts.flatten())

In [15]:
poly_consts

array([ 2.29645346e-02,  5.00000000e-01,  1.43401235e+00,  1.09660623e-10,
       -2.08918475e+00, -3.41051184e-10,  1.15517240e+00,  2.31396971e-10])

In [16]:
poly_consts.dtype

dtype('float64')